In [100]:
from pynq.overlays.base import BaseOverlay 
import time
from datetime import datetime

In [101]:
base = BaseOverlay("base.bit") 
btns = base.btns_gpio

In [102]:
%%microblaze base.PMODA 
#include "gpio.h"
#include "pyprintf.h"
//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}
//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

In [103]:

import asyncio 
cond = True 
pin = 0 #start on red

async def pwm(frequency, duty_cycle): 
    global cond, pin
    
    period = 1/frequency
    on = duty_cycle/100 * period
    off = (100-duty_cycle)/100 * period
    
    print("entering loop")
    while cond:
        if pin == 0: # blink red
            write_gpio(3, 1)
            await asyncio.sleep(on) 
            write_gpio(3, 0)
            await asyncio.sleep(off)
            
        if pin == 1: # blinking green led 
            write_gpio(2, 1)
            await asyncio.sleep(on) 
            write_gpio(2, 0)
            await asyncio.sleep(off)
            
        if pin == 2: # blinking blue led 
            write_gpio(1, 1)
            await asyncio.sleep(on) 
            write_gpio(1, 0)
            await asyncio.sleep(off)
            
            
async def get_btns(_loop): 
    global cond, pin 
    
    while cond:
        await asyncio.sleep(0.01) 
        
        if btns[0].read() != 0: # blinking red led
            pin = 0
            await asyncio.sleep(0.3)
            print("Pressed btn 0")
            
        elif btns[1].read() != 0: # blinking green led 
            pin = 1
            await asyncio.sleep(0.3)
            print("Pressed btn 1")
            
        elif btns[2].read() != 0: # blinking blue led 
            pin = 2
            await asyncio.sleep(0.3)
            print("Pressed btn 2")
            
        elif btns[3].read() != 0: #stop blinking 
            cond = False
            await asyncio.sleep(0.3)
            print("Pressed btn 3")
            clear_gpio()
            _loop.stop()
            
def clear_gpio():
    write_gpio(7, 0)
    write_gpio(6, 0)
    write_gpio(5, 0)
    write_gpio(4, 0)
    write_gpio(3, 0)
    write_gpio(2, 0)
    write_gpio(1, 0)

In [104]:
print("Start")
clear_gpio()
loop = asyncio.new_event_loop() 

write_gpio(3,1)
await asyncio.sleep(1)
write_gpio(3,0)
await asyncio.sleep(0)

freq, duty = 10, 25
loop.create_task(pwm(freq, duty)) 
loop.create_task(get_btns(loop)) 
loop.run_forever()
loop.close()

print("Done")

Start
entering loop
Pressed btn 2
Pressed btn 1
Pressed btn 0
Pressed btn 3
Done
